##Mount Google Drive

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


##Set up root

In [2]:
# Specify the directory path
root = '/gdrive/MyDrive/Irfan/models/'
!ls '/gdrive/MyDrive/Irfan/models/'

'a scatch.gdoc'   h5   ipynb   model_graph.png	 stack.best.hdf5


##Import Libraries

In [3]:
from sklearn.metrics import accuracy_score
from keras.models import load_model
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from numpy import argmax

from keras import metrics, optimizers
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.applications.vgg16 import preprocess_input
from matplotlib import pyplot as plt
import numpy as np
import json as js
import tensorflow as tf

##Set parameter value

In [4]:
batch_size = 32
epochs = 50

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

nb_training_samples = 1600
nb_validation_samples = 400# Set parameter values

##Data Loader

In [5]:
data = '/gdrive/MyDrive/Irfan/ESC-50-master'

training_data_dir = data + '/audio augmented/training'
validation_data_dir = data + '/audio augmented/testing/'

training_datagen = image.ImageDataGenerator(
    rescale=1./255)

validation_datagen = image.ImageDataGenerator(
    rescale=1./255)

def generator_copier(generator, dir, batch_size, img_height, img_width):
  gen = generator.flow_from_directory(dir,
                                      target_size=(img_height, img_width),
                                      batch_size=batch_size)
  
  while True:
    x = gen.next()
    yield [x[0] for _ in range(5)], x[1]


# training generator configuration
training_generator = generator_copier(generator=training_datagen,
                                           dir=training_data_dir,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)


# validation generator configuration
validation_generator = generator_copier(generator=validation_datagen,
                                          dir=validation_data_dir,
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_width)

## load models from file


In [6]:
def top_5_accuracy(y_true, y_pred):
      return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)

In [7]:
def load_all_models():
  models_name = ['DenseNet201', 'Xception', 'VGG19', 'ResNet152V2', 'InceptionV3']
  all_models = []
  for i in range(5):
    # define filename for this ensemble
    filename = root + 'h5/' + models_name[i] + '.h5'
		# load model from file
    model = load_model(filename, custom_objects={'top_5_accuracy': top_5_accuracy})
    # add to list of members
    all_models.append(model)
    print('>loaded %s' % filename)
  return all_models

In [8]:
num_layers_to_freeze = 0

In [9]:
def define_stacked_model(members):
  # update all layers in all models to not be trainable
  for i in range(len(members)):
    model = members[i]
    for layer in model.layers:
      # make not trainable
      layer.trainable = False
      # rename to avoid 'unique layer name' issue
      layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
      global num_layers_to_freeze
      num_layers_to_freeze += 1
  # define multi-headed input
  ensemble_visible = [model.input for model in members]
  # concatenate merge output from each model
  ensemble_outputs = [model.output for model in members]
  merge = concatenate(ensemble_outputs)
  hidden = Dense(128, activation='relu')(merge)
  droput = Dropout(0.5)(hidden)
  output = Dense(50, activation='softmax')(droput)
  model = Model(inputs=ensemble_visible, outputs=output)
  # plot graph of ensemble
  graph_file = root + 'model_graph.png'
  plot_model(model, show_shapes=True, to_file=graph_file)
  # compile
  model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), metrics=['accuracy', top_5_accuracy])
  return model

##stacked pipeline

In [10]:
models = load_all_models()

>loaded /gdrive/MyDrive/Irfan/models/h5/DenseNet201.h5
>loaded /gdrive/MyDrive/Irfan/models/h5/Xception.h5
>loaded /gdrive/MyDrive/Irfan/models/h5/VGG19.h5
>loaded /gdrive/MyDrive/Irfan/models/h5/ResNet152V2.h5
>loaded /gdrive/MyDrive/Irfan/models/h5/InceptionV3.h5


In [11]:
stacked = define_stacked_model(models)
#stacked.summary()

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.416052 to fit



##Training Pipeline

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from time import time

tensorboard = TensorBoard(log_dir="logs/layers_frozen_{}".format(num_layers_to_freeze))

# checkpoint
filepath= root + "/stack.best.hdf5" #CHANGE THIS TOO
best_model_checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [best_model_checkpoint, tensorboard]

history = stacked.fit(
    training_generator,
    steps_per_epoch=nb_training_samples/batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size,
    callbacks=callbacks_list)

#plotting accuracy
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.title(str(model) + 'accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plotting loss
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title(str(model) + 'loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Found 4800 images belonging to 50 classes.
Epoch 1/50
50/50 [==============================] - 3094s 61s/step - loss: 3.9473 - accuracy: 0.0080 - top_5_accuracy: 0.0677 - val_loss: 3.9352 - val_accuracy: 0.0000e+00 - val_top_5_accuracy: 0.0409

Epoch 00001: val_accuracy improved from -inf to 0.00000, saving model to /gdrive/MyDrive/Irfan/models/stack.best.hdf5
Epoch 2/50
50/50 [==============================] - 3067s 61s/step - loss: 3.9522 - accuracy: 0.0082 - top_5_accuracy: 0.0662 - val_loss: 3.9297 - val_accuracy: 0.0048 - val_top_5_accuracy: 0.0625

Epoch 00002: val_accuracy improved from 0.00000 to 0.00481, saving model to /gdrive/MyDrive/Irfan/models/stack.best.hdf5
Epoch 3/50
50/50 [==============================] - 3038s 61s/step - loss: 3.9507 - accuracy: 0.0099 - top_5_accuracy: 0.0692 - val_loss: 3.9235 - val_accuracy: 0.0050 - val_top_5_accuracy: 0.0450

Epoch 00003: val_accuracy improved from 0.00481 to 0.00500, saving model to /gdrive/MyDrive/Irfan/models/stack.best.hdf5